In [36]:
#import pyodbc
import pymysql
import codecs
import pandas as pd

import socket

from datetime import datetime
import threading

now = datetime.now()

# Mysql DB CONNECT, INSERT,  SELECT
def db_connect_mysql():
    host = ""
    port = 3306
    database = ""
    username = ""
    password = ""
    try: 
        conn =  pymysql.connect(host, user=username, passwd=password, db=database, port=port, use_unicode=True, charset='utf8')
    except BaseException:
        print("DB connection Error") 
    cursor = conn.cursor()  
    
    return conn, cursor

# Tibrero DB CONNECT, INSERT,  SELECT
def db_connect_tibero():
    try: 
        conn =  pyodbc.connect()
    except BaseException:
        print("DB connection Error") 
    cursor = conn.cursor()  
    
    return conn, cursor


# 도시 아파트 코드 API 호출
def db_insert_batch(sql, APT_CODE, APT_NAME, SIDO_CODE, CRT_DTHR, CRT_IP_MAC):
    conn, cursor = db_connect()
    cursor.execute(sql, (APT_CODE, APT_NAME, SIDO_CODE, CRT_DTHR, CRT_IP_MAC))
    conn.commit()
    conn.close()
    
# 아파트 기본정보 API 호출
def db_update_batch(sql, apt_code, addr, household_cnt, is_addr_error, is_lon_error):
    conn, cursor = db_connect()
    with conn: # with 절 종료 시점에 db 닫음
        with conn.cursor() as cur:
            cur.execute(sql, (apt_code, addr, household_cnt, is_addr_error, is_lon_error))

# 결과
def db_select(sql):
    conn, cursor = db_connect()
    cursor.execute(sql)
    result = pd.DataFrame(cursor.fetchall())   
    conn.close()
    
    return result

# open api를 호출해 도시의 아파트 목록 xml 데이터를 획득하는 함수입니다
def xml_sido_Apt(serviceUrl, sidoCode, pageNo, numOfRows, serviceKey=''): 
    from urllib.request import urlopen
    
    max_retries = 4
    retry_delay = 10  # seconds
 
    url = serviceUrl + '?sidoCode=' + sidoCode + '&pageNo=' + pageNo + '&numOfRows=' + numOfRows + '&serviceKey=' + serviceKey           
    
    for attempt in range(max_retries):
        try:
            result = urlopen(url, timeout=5)  # request 요청 
            xmlobj = BeautifulSoup(result, 'lxml-xml') # xml 데이터 획득
            
            break  # Exit the loop if the request succeeds
            
        except (urllib.error.URLError):
            
            if attempt < max_retries - 1:
                print(f'Retrying in {retry_delay} seconds...')
                time.sleep(retry_delay)
            else:
                print('Maximum retries exceeded')    
    
    return xmlobj



# 시스템에 접속 된 호스트 이름
print(socket.gethostbyname(socket.gethostname()))


# 현재 시간
print("문자열 변환 : ", now.strftime('%Y/%m/%d'))

192.168.134.1
문자열 변환 :  2023/02/20


In [37]:
# open api를 호출해 전국 아파트 목록 xml 데이터를 획득하는 함수입니다
def xml_total_Apt(serviceUrl, pageNo, numOfRows, serviceKey=''): 
    from urllib.request import urlopen
    import time
    from bs4 import BeautifulSoup # document parser
    import xml.etree.ElementTree as elemTree
    
    max_retries = 4
    retry_delay = 10  # seconds
 
    url = serviceUrl + '?pageNo=' + pageNo + '&numOfRows=' + numOfRows + '&serviceKey=' + serviceKey           
    
    for attempt in range(max_retries):
        try:
            result = urlopen(url)  # request 요청 
            xmlobj = BeautifulSoup(result, 'lxml-xml') # xml 데이터 획득
            
            break  # Exit the loop if the request succeeds
            
        except (urllib.error.URLError):
            
            if attempt < max_retries - 1:
                print(f'Retrying in {retry_delay} seconds...')
                time.sleep(retry_delay)
            else:
                print('Maximum retries exceeded')    
    
    return xmlobj


In [12]:
xmlDoc

<?xml version="1.0" encoding="utf-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><as1>경상남도</as1><as2>창원마산회원구</as2><as3>내서읍</as3><as4>삼계리</as4><bjdCode>4812725028</bjdCode><kaptCode>A63085232</kaptCode><kaptName>마산삼계2</kaptName></item><item><as1>부산광역시</as1><as2>연제구</as2><as3>연산동</as3><bjdCode>2647010200</bjdCode><kaptCode>A61108001</kaptCode><kaptName>연산센트럴파크</kaptName></item><item><as1>울산광역시</as1><as2>북구</as2><as3>중산동</as3><bjdCode>3120010600</bjdCode><kaptCode>A68380902</kaptCode><kaptName>이화제일그린파크</kaptName></item><item><as1>경기도</as1><as2>평택시</as2><as3>안중읍</as3><as4>현화리</as4><bjdCode>4122025330</bjdCode><kaptCode>A45176505</kaptCode><kaptName>안중현대3차</kaptName></item><item><as1>전라남도</as1><as2>순천시</as2><as3>조례동</as3><bjdCode>4615013300</bjdCode><kaptCode>A54032002</kaptCode><kaptName>금당대림아파트</kaptName></item><item><as1>대구광역시</as1><as2>달서구</as2><as3>상인동</as3><bjdCode>2729011500</bjdCode><kaptCode>A70437018</kaptCode><k

# 전국 아파트 단지 코드 조회

In [40]:
def setTotalAptList(): 
    crt_host = socket.gethostbyname(socket.gethostname())
    crt_dthr = now.strftime('%Y/%m/%d')
    
    # xml 데이터 획득
    xmlDoc = xml_total_Apt('http://apis.data.go.kr/1613000/AptListService2/getTotalAptList','1','10')
    print(xmlDoc)
    resultCode = xmlDoc.find('resultCode')  # 결과 상태 코드 확인
    
    sql = "INSERT INTO CMN_APT_BATCH (APT_CODE, APT_NAME, SIDO_CODE, CRT_DTHR, CRT_IP_MAC, CRT_MENU_ID, CRT_MENU_SEQ) VALUES(%s, %s, %s, %s, %s, 'Mymap', 20230208144520437681)"
    sql2 = "SELECT * FROM CMN_APT_BATCH"
    
    if resultCode is not None: # 데이터가 있는 경우
        totalCount = xmlDoc.find('totalCount').get_text() # 전체 데이터 개수 확인
        numOfRows = xmlDoc.find('numOfRows').get_text()  # 한 페이지 당 출력 데이터 개수 확인

        totalCount = int(totalCount)
        numOfRows = int(numOfRows)

        loopCount = round(totalCount/numOfRows) # 총 open api 호출 횟수 소숫점 첫째자리에서 바로 반올림
        
        if ((loopCount * numOfRows) < totalCount): # 총 반복 횟수 보정 
            loopCount = loopCount + 1
        
        for pageNo in range(1, loopCount):  #  페이지 반복문
            # xml 데이터 획득
            xmlDoc = xml_total_Apt('http://apis.data.go.kr/1613000/AptListService2/getTotalAptList', str(pageNo),'10')
            print("현재 페이지 번호입니다 >>> ", pageNo)
            data = xmlDoc.findAll('item')  # xml에서 데이터 요소 찾기
            if data is not None: # 데이터가 있는 경우
                print(data)
                for item in data:
                    as1 = item.find('as1')
                    apt_code = item.find('kaptCode')
                    apt_name = item.find('kaptName')

                    if as1 is not None :
                        if "서울" in as1.get_text():
                            sido_code= "11"

                        elif "부산" in as1.get_text():
                            sido_code= "26"

                        elif "대구" in as1.get_text():
                            sido_code= "27"

                        elif "인천" in as1.get_text():
                            sido_code= "28"

                        elif "광주" in as1.get_text():
                            sido_code= "29"

                        elif "대전" in as1.get_text():
                            sido_code= "30"

                        elif "울산" in as1.get_text():
                            sido_code= "31"

                        elif "경기" in as1.get_text():
                            sido_code= "41"

                        elif "강원" in as1.get_text():
                            sido_code= "42"

                        elif "충청남도" in as1.get_text():
                            sido_code= "44"

                        elif "충청북도" in as1.get_text():
                            sido_code= "43"

                        elif "전라북도" in as1.get_text():
                            sido_code= "45"

                        elif "전라남도" in as1.get_text():
                            sido_code= "46"

                        elif "경상북도" in as1.get_text():
                            sido_code= "47"

                        elif "경상남도" in as1.get_text():
                            sido_code= "48"

                        elif "제주" in as1.get_text():
                            sido_code= "50"

                        else: 
                            sido_code="-1"

                    if apt_code is not None:
                        aptCode = apt_code.get_text()

                    if apt_name is not None:
                        aptName = apt_name.get_text()

                    print(aptCode, aptName, sido_code, crt_dthr, crt_host)
                    db_insert_batch(sql, aptCode, aptName, sido_code, crt_dthr, crt_host)  # 저장

                    df= db_select(sql2) # 결과
                    print("결과")
                    print(df)
           

            else: # 데이터가 없는 경우
                print("Data not found.")       